Notas de aula para o concurso para professor adjunto... edital blablabla. 

# Definições

In [55]:
from sympy import *
from itertools import *

# Einsteinpy: https://docs.einsteinpy.org/en/stable/index.html
# Sympy: https://docs.sympy.org/latest/index.html
# Intertools: https://docs.python.org/3/library/itertools.html

In [56]:
# Respectivamente: Massa M, constante de Newton G, velocidade da luz c, constante cosmológica Lb tempo t, raio r, angulo polar theta, azimuth phi 

M, G, c, Lb, t, r, th, ph = symbols('M G c Lb t r th ph')
X = [t, r, th, ph]

# Também definimos as funções f1 e f2 que usamos no ansätz para a solução de Schwarzschild
f1 = Function('f1')('r')
f2 = Function('f2')('r')

# Ansätz

A equação de campo de Einstein no vácuo, mas na presença de uma constante cosmológica, é dada por: 

$$ \tag{1} R_{\mu\nu} - \frac{1}{2} (R - 2 \Lambda) g_{\mu\nu} = 0 \qquad (\mu, \nu = 0, 1, 2, 3) \quad \Rightarrow \quad \boxed{R_{\mu\nu} = \Lambda g_{\mu\nu}}$$

onde $R_{\mu\nu}$ são as componentes do tensor de Ricci, $R$ o escalar de curvatura, $\Lambda$ a constante cosmológica e $g_{\mu\nu}$ as componentes da métrica, de tal forma que o elemento de distântia $ds^2$ será dado por 
$$ \tag{2} ds^2 = \sum_{\mu=0}^3 \sum_{\nu=0}^3 g_{\mu\nu} dx^\mu dx^\nu  \equiv  g_{\mu\nu} dx^\mu dx^\nu $$
onde na última igualdade usamos a convenção de Einstein para soma. Buscaremos uma solução de (1) tal que:

1) Possuem simetria esférica. Podemos pensar nessa condição como o equivalente Newtoniano de um potencial central. De fato, estudando a equação da geodésica para a solução de Schwarzschild (que encontraremos abaixo) nos levará ao potencial $\Phi(r) = - \frac{GM}{r}$.

2) O espaço tempo é estático, ou seja, as componentes da métrica são independentes do tempo, e em particular, consideraremos a invariancia sob a transformação $t \mapsto -t$.

3) O espaço-tempo é assimptoticamente plano.

Essas 3 condições juntas nos induzem a considerar um ansätz na forma
$$\tag{3} ds^2 = -f_1(r) c^2 dt^2 + f_2(r)dr^2 + r^2 (d\theta^2 + \sin^2\theta d\phi^2)$$

## Resolvendo a equação de Einstein

Vamos agora resolver a solução de Einstein passo a passo usando o ansätz dado na expressão (3).

### Métrica

In [57]:
# Aqui vamos definir a métrica. Inicialmente preenchemos todas as entradas com zeros, e depois definimos as componentes não nulas. 
g = zeros(4) 
g[0,0] = -f1 # felizmente o python é relativista e começa a contar os indices a partir do zero 
g[1,1] = f2
g[2,2] = r**2
g[3,3] = r**2 * sin(th)**2

rank = g.rank() # o rank será útil para generalizar esse código para casos onde o espaço-tempo tem dimensões maiores ou menos que 4. Sim, isso é muito importante. 
gI = g.inv() # Também precisamos da inversa para subir os indices de Lorentz

### Símbolos de Christoffel

Esses objetos são as componentes da conexão de Levi-Civita dadas explicitamente por:

$$\tag{4} \Gamma^\rho_{\mu\nu} = \frac{1}{2} g^{\rho\sigma} \left( \partial_\mu g_{\sigma\nu} + \partial_\sigma g_{\mu\nu} - \partial_\sigma g_{\mu\nu} \right) $$

In [58]:
# k denota os indices contravariantes (no andar de cima), ao passo que i e j denotam os indices covariantes (no andar de baixo).
def Gamma(k,i,j):
    n = 0
    GammaMatrix = 0
    while n < rank:
        GammaMatrix = GammaMatrix + gI[k,n]*(g[n,j].diff(X[i]) + g[n,i].diff(X[j]) - g[i,j].diff(X[n])) / 2
        n += 1
    return GammaMatrix

É útil pensar nesses objetos, os símbolos de Christoffel como 4 matrizes rotulados com o indice k = 0, 1, 2, 3, de tal forma que:

In [59]:
# Primeiro construímos 4 matrizes com todas as componentes nulas
ChSymb0 = zeros(rank)
ChSymb1 = zeros(rank)
ChSymb2 = zeros(rank)
ChSymb3 = zeros(rank)
ChSymb = [ChSymb0, ChSymb1, ChSymb2, ChSymb3]

In [60]:
# Agora preenchemos essas matrizes com as componentes não nulas para o ansätz dado em (3)
for k in range(rank):
    for i in range(rank):
        for j in range(rank):
            if Gamma(k,i,j) != 0:
                ChSymb[k][i,j] = Gamma(k,i,j)

In [61]:
ChSymb[0]

Matrix([
[                             0, Derivative(f1(r), r)/(2*f1(r)), 0, 0],
[Derivative(f1(r), r)/(2*f1(r)),                              0, 0, 0],
[                             0,                              0, 0, 0],
[                             0,                              0, 0, 0]])

In [62]:
ChSymb[1]

Matrix([
[Derivative(f1(r), r)/(2*f2(r)),                              0,        0,                   0],
[                             0, Derivative(f2(r), r)/(2*f2(r)),        0,                   0],
[                             0,                              0, -r/f2(r),                   0],
[                             0,                              0,        0, -r*sin(th)**2/f2(r)]])

In [63]:
ChSymb[2]

Matrix([
[0,   0,   0,                0],
[0,   0, 1/r,                0],
[0, 1/r,   0,                0],
[0,   0,   0, -sin(th)*cos(th)]])

In [64]:
ChSymb[3]

Matrix([
[0,   0,               0,               0],
[0,   0,               0,             1/r],
[0,   0,               0, cos(th)/sin(th)],
[0, 1/r, cos(th)/sin(th),               0]])

### Tensor de Riemann

De posse dos simbolos de Christoffel, podemos encontrar o tensor de Riemann pela expressão

$$\tag{5} R^l_{kij} = \partial_i \Gamma^l_{kj} - \partial_j \Gamma^l_{ki} + \Gamma^l_{ip}\Gamma^p_{kj} -  \Gamma^l_{jp}\Gamma^p_{ki} $$

In [65]:
# l é contravariante; k,i,j são covariantes
# Aqui vamos definir a constração do produto de dois simbolos de Christoffel. Ou seja, os dois últimos termos na equação (5)
def Contraction(l,k,i,j):
    n = 0 
    cont = 0 
    while n < rank:
        cont = cont + ChSymb[l][i,n] * ChSymb[n][k,j] - ChSymb[l][j,n] * ChSymb[n][k,i]
        n += 1
    return cont

In [66]:
# Agora calculamos as componentes do tensor de Riemann propriamente ditas. 
def Riem(l,k,i,j):
    n = 0
    Riemann = 0
    while n < rank:
        Riemann = Riemann + ChSymb[l][k,j].diff(X[i]) - ChSymb[l][k,i].diff(X[j]) + Contraction(l,k,i,j)
        n += 1
    R = Riemann.simplify()
    return R

Com as componentes que podemos calcular usando a função acima, vamos organizar esses resultados em termos de matrizes $(R^l_k)_{ij}$

In [67]:
# Mais uma vez, definimos as matrizes e preenchemos com zeros. 
RieTen00 = zeros(rank); RieTen01 = zeros(rank); RieTen02 = zeros(rank); RieTen03 = zeros(rank)
RieTen10 = zeros(rank); RieTen11 = zeros(rank); RieTen12 = zeros(rank); RieTen13 = zeros(rank)
RieTen20 = zeros(rank); RieTen21 = zeros(rank); RieTen22 = zeros(rank); RieTen23 = zeros(rank)
RieTen30 = zeros(rank); RieTen31 = zeros(rank); RieTen32 = zeros(rank); RieTen33 = zeros(rank)

Rie = [[RieTen00, RieTen01, RieTen02, RieTen03], 
       [RieTen10, RieTen11, RieTen12, RieTen13],
       [RieTen20, RieTen21, RieTen22, RieTen23],
       [RieTen30, RieTen31, RieTen32, RieTen33]]

In [68]:
# E agora podemos preencher as componentes não nulas nas matrizes acima
for l in range(rank):
    for k in range(rank):
        for i in range(rank):
            for j in range(rank):
                if Riem(l,k,i,j) != 0:
                    Rie[l][k][i,j] = Riem(l,k,i,j)

In [69]:
# Para ver o resultado explícito, fazemos
for l in range(rank):
    for k in range(rank):
        display(Rie[l][k])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[                                                                                                                             0, -2*Derivative(f1(r), (r, 2))/f1(r) + Derivative(f1(r), r)*Derivative(f2(r), r)/(f1(r)*f2(r)) + Derivative(f1(r), r)**2/f1(r)**2, 0, 0],
[2*Derivative(f1(r), (r, 2))/f1(r) - Derivative(f1(r), r)*Derivative(f2(r), r)/(f1(r)*f2(r)) - Derivative(f1(r), r)**2/f1(r)**2,                                                                                                                               0, 0, 0],
[                                                                                                                             0,                                                                                                                               0, 0, 0],
[                                                                                                                             0,                                                                    

Matrix([
[                                     0, 0, -2*r*Derivative(f1(r), r)/(f1(r)*f2(r)), 0],
[                                     0, 0,                                       0, 0],
[2*r*Derivative(f1(r), r)/(f1(r)*f2(r)), 0,                                       0, 0],
[                                     0, 0,                                       0, 0]])

Matrix([
[                                                0, 0, 0, -2*r*sin(th)**2*Derivative(f1(r), r)/(f1(r)*f2(r))],
[                                                0, 0, 0,                                                  0],
[                                                0, 0, 0,                                                  0],
[2*r*sin(th)**2*Derivative(f1(r), r)/(f1(r)*f2(r)), 0, 0,                                                  0]])

Matrix([
[                                                                                                                             0, -2*Derivative(f1(r), (r, 2))/f2(r) + Derivative(f1(r), r)*Derivative(f2(r), r)/f2(r)**2 + Derivative(f1(r), r)**2/(f1(r)*f2(r)), 0, 0],
[2*Derivative(f1(r), (r, 2))/f2(r) - Derivative(f1(r), r)*Derivative(f2(r), r)/f2(r)**2 - Derivative(f1(r), r)**2/(f1(r)*f2(r)),                                                                                                                               0, 0, 0],
[                                                                                                                             0,                                                                                                                               0, 0, 0],
[                                                                                                                             0,                                                                    

...

Matrix([
[0, 0,                              0,                              0],
[0, 0,                              0,                              0],
[0, 0,                              0, 4*(f2(r) - 1)*sin(th)**2/f2(r)],
[0, 0, 4*(1 - f2(r))*sin(th)**2/f2(r),                              0]])

Matrix([
[                               0, 0, 0, -2*Derivative(f1(r), r)/(r*f2(r))],
[                               0, 0, 0,                                 0],
[                               0, 0, 0,                                 0],
[2*Derivative(f1(r), r)/(r*f2(r)), 0, 0,                                 0]])

Matrix([
[0,                                0, 0,                                 0],
[0,                                0, 0, -2*Derivative(f2(r), r)/(r*f2(r))],
[0,                                0, 0,                                 0],
[0, 2*Derivative(f2(r), r)/(r*f2(r)), 0,                                 0]])

Matrix([
[0, 0,           0,            0],
[0, 0,           0,            0],
[0, 0,           0, -4 + 4/f2(r)],
[0, 0, 4 - 4/f2(r),            0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### Tensor de Ricci

Finalmente, podemos calcular o tensor de Ricci como 
$$\tag{6} R_{ij} = R^k_{ikj}$$

In [70]:
def Ricci(i,j):
    ricc = 0
    n = 0
    for k in range(rank):
        ricc = ricc + Rie[k][i][k,j]
        r = ricc.simplify()
    return r

In [71]:
Ric = [Ricci(i,j).simplify() for i in range(rank) for j in range(rank) if Ricci(i,j) != 0]

In [72]:
len(Ric)

4

In [85]:
# Para ver o resultado explícito, fazemos
for k in range(len(Ric)):
    print(f"+++++++++ Ricci({k},{k}) ++++++++++++++++")
    display(Ric[k])
    print(" ")

+++++++++ Ricci(0,0) ++++++++++++++++


2*Derivative(f1(r), (r, 2))/f2(r) - Derivative(f1(r), r)*Derivative(f2(r), r)/f2(r)**2 - Derivative(f1(r), r)**2/(f1(r)*f2(r)) + 4*Derivative(f1(r), r)/(r*f2(r))

 
+++++++++ Ricci(1,1) ++++++++++++++++


-2*Derivative(f1(r), (r, 2))/f1(r) + Derivative(f1(r), r)*Derivative(f2(r), r)/(f1(r)*f2(r)) + Derivative(f1(r), r)**2/f1(r)**2 + 4*Derivative(f2(r), r)/(r*f2(r))

 
+++++++++ Ricci(2,2) ++++++++++++++++


2*r*Derivative(f2(r), r)/f2(r)**2 - 2*r*Derivative(f1(r), r)/(f1(r)*f2(r)) + 4 - 4/f2(r)

 
+++++++++ Ricci(3,3) ++++++++++++++++


2*(r*f1(r)*Derivative(f2(r), r) - r*f2(r)*Derivative(f1(r), r) + 2*(f2(r) - 1)*f1(r)*f2(r))*sin(th)**2/(f1(r)*f2(r)**2)

### Análise dos resultados

1) Primeiro, observe os tensores de Ricci que acabamos de calcular devem ser nulos pela equação de campo de Einstein (1). Além disso, apenas 3 equações são independentes. É simples verificar que 
$$\tag{7} R_{33} - sin^2\theta R_{2,2} = 0 $$

In [81]:
(Ric[3] - Ric[2]*(sin(th)**2)).simplify()

0

Então podemos concentrar nossa atenção nas 3 primeiras equações

2. Observe agora que as 2 primeiras equações possuem uma estrutura muito semelhante, então faremos a operação a seguir 

$$ \tag{7} f_1 f_2^2 R_{00} - f_1^2 f_2 R_{11} = 0 $$

In [86]:
((f1*f2**2)*Ric[0] + (f2*f1**2)*Ric[1]).simplify()

4*(f1(r)*Derivative(f2(r), r) + f2(r)*Derivative(f1(r), r))*f1(r)/r

de onde concluímos que 
$$  \tag{8} \frac{d}{dr}(f_1 f_2) = 0 \quad \Rightarrow \quad f_2 = \frac{c_0}{f_1}  $$
onde $c_0$ é uma constante. 

3. Agora vamos usar essa condição na equação 
$$ \tag{8} R_{22} = 0 $$

In [92]:
f2 = (f1)**(-1)

In [100]:
(Ric[2]).simplify()

2*r*Derivative(f2(r), r)/f2(r)**2 - 2*r*Derivative(f1(r), r)/(f1(r)*f2(r)) + 4 - 4/f2(r)

In [110]:
f1 = 1 - G*M*(r**(-1))
f2 = (1 - G*M*(r**(-1)))**(-1)

In [111]:
((2*r*f2.diff(r))*(f2**(-2)) - (2*r*f1.diff(r))*(f1*f2) + 4 - 4*(f2)**(-1)).simplify()

0

# Solução de Schwarzschild

## Metrica

Comecaremos com a metrica

$$ g \equiv ds^2 = -\left(1 - \frac{2 G M}{c^2 r}\right) c^2 dt^2 + \frac{dr^2}{\left(1 - \frac{2 G M}{c^2 r}\right)} + r^2 d\theta^2 + r^2 \sin^2\theta d\phi^2 $$

In [161]:
# vamos definir a funcao f 
f =  1 - 2*M*G / (c**2 * r)
g = zeros(4)

In [162]:
g[0,0] = - f
g[1,1] = 1/f
g[2,2] = r**2
g[3,3] = r**2 * sin(th)**2
rank = g.rank()

In [163]:
# Inversa 
gI = g.inv()

In [164]:
g

Matrix([
[2*G*M/(c**2*r) - 1,                       0,    0,               0],
[                 0, 1/(-2*G*M/(c**2*r) + 1),    0,               0],
[                 0,                       0, r**2,               0],
[                 0,                       0,    0, r**2*sin(th)**2]])

## Simbolos de Christoffel

In [165]:
# k is the contravariant (upper) index, while i and j are covariant (lower) indices

def Gamma(k,i,j):
    n = 0
    GammaMatrix = 0
    while n < rank:
        GammaMatrix = GammaMatrix + gI[k,n]*(g[n,j].diff(X[i]) + g[n,i].diff(X[j]) - g[i,j].diff(X[n])) / 2
        n += 1
    return GammaMatrix

It's useful to think of these objects as 4 matrices labeled by the integer k
$$ (\Gamma^0)_{ij} \quad (\Gamma^1)_{ij} \quad (\Gamma^2)_{ij} \quad (\Gamma^3)_{ij} $$

In [166]:
ChSymb0 = zeros(rank)
ChSymb1 = zeros(rank)
ChSymb2 = zeros(rank)
ChSymb3 = zeros(rank)

In [167]:
ChSymb = [ChSymb0, ChSymb1, ChSymb2, ChSymb3]

In [168]:
for k in range(rank):
    for i in range(rank):
        for j in range(rank):
            if Gamma(k,i,j) != 0:
                ChSymb[k][i,j] = Gamma(k,i,j)

In [169]:
ChSymb[0]

Matrix([
[                        0, G*M/(r*(-2*G*M + c**2*r)), 0, 0],
[G*M/(r*(-2*G*M + c**2*r)),                         0, 0, 0],
[                        0,                         0, 0, 0],
[                        0,                         0, 0, 0]])

In [170]:
ChSymb[1]

Matrix([
[G*M*(-2*G*M + c**2*r)/(c**4*r**3),                                                           0,                       0,                                  0],
[                                0, -G*M*(-2*G*M + c**2*r)/(c**4*r**3*(-2*G*M/(c**2*r) + 1)**2),                       0,                                  0],
[                                0,                                                           0, -(-2*G*M + c**2*r)/c**2,                                  0],
[                                0,                                                           0,                       0, -(-2*G*M + c**2*r)*sin(th)**2/c**2]])

In [171]:
ChSymb[2]

Matrix([
[0,   0,   0,                0],
[0,   0, 1/r,                0],
[0, 1/r,   0,                0],
[0,   0,   0, -sin(th)*cos(th)]])

In [172]:
ChSymb[3]

Matrix([
[0,   0,               0,               0],
[0,   0,               0,             1/r],
[0,   0,               0, cos(th)/sin(th)],
[0, 1/r, cos(th)/sin(th),               0]])

## Riemann Tensor

In [173]:
# l is the contravariant index, k,i,j are covariant indices
# I'm gonna use the objects ChSymb because I don't want to compute these symbols at each run
def Contraction(l,k,i,j):
    n = 0 
    cont = 0 
    while n < rank:
        cont = cont + ChSymb[l][i,n] * ChSymb[n][k,j] - ChSymb[l][j,n] * ChSymb[n][k,i]
        n += 1
    return cont

In [174]:
def Riem(l,k,i,j):
    n = 0
    Riemann = 0
    while n < rank:
        Riemann = Riemann + ChSymb[l][k,j].diff(X[i]) - ChSymb[l][k,i].diff(X[j]) + Contraction(l,k,i,j)
        n += 1
    R = Riemann.simplify()
    return R

Now we can finally calculate the components of the Riemann tensor. It is useful to consider the components as a matrix of matrices $(R^l_k)_{ij}$

In [175]:
RieTen00 = zeros(rank); RieTen01 = zeros(rank); RieTen02 = zeros(rank); RieTen03 = zeros(rank)
RieTen10 = zeros(rank); RieTen11 = zeros(rank); RieTen12 = zeros(rank); RieTen13 = zeros(rank)
RieTen20 = zeros(rank); RieTen21 = zeros(rank); RieTen22 = zeros(rank); RieTen23 = zeros(rank)
RieTen30 = zeros(rank); RieTen31 = zeros(rank); RieTen32 = zeros(rank); RieTen33 = zeros(rank)

In [176]:
Rie = [[RieTen00, RieTen01, RieTen02, RieTen03], 
       [RieTen10, RieTen11, RieTen12, RieTen13],
       [RieTen20, RieTen21, RieTen22, RieTen23],
       [RieTen30, RieTen31, RieTen32, RieTen33]]

In [177]:
for l in range(rank):
    for k in range(rank):
        for i in range(rank):
            for j in range(rank):
                if Riem(l,k,i,j) != 0:
                    Rie[l][k][i,j] = Riem(l,k,i,j)

In [178]:
# Run this cell to see the matrices explicitly
for l in range(rank):
    for k in range(rank):
        display(Rie[l][k])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[                            0, 8*G*M/(r**2*(-2*G*M + c**2*r)), 0, 0],
[8*G*M/(r**2*(2*G*M - c**2*r)),                              0, 0, 0],
[                            0,                              0, 0, 0],
[                            0,                              0, 0, 0]])

Matrix([
[             0, 0, -4*G*M/(c**2*r), 0],
[             0, 0,               0, 0],
[4*G*M/(c**2*r), 0,               0, 0],
[             0, 0,               0, 0]])

Matrix([
[                        0, 0, 0, -4*G*M*sin(th)**2/(c**2*r)],
[                        0, 0, 0,                          0],
[                        0, 0, 0,                          0],
[4*G*M*sin(th)**2/(c**2*r), 0, 0,                          0]])

Matrix([
[                                 0, 8*G*M*(-2*G*M + c**2*r)/(c**4*r**4), 0, 0],
[8*G*M*(2*G*M - c**2*r)/(c**4*r**4),                                   0, 0, 0],
[                                 0,                                   0, 0, 0],
[                                 0,                                   0, 0, 0]])

...

Matrix([
[0, 0,                          0,                         0],
[0, 0,                          0,                         0],
[0, 0,                          0, 8*G*M*sin(th)**2/(c**2*r)],
[0, 0, -8*G*M*sin(th)**2/(c**2*r),                         0]])

Matrix([
[                                  0, 0, 0, 4*G*M*(2*G*M - c**2*r)/(c**4*r**4)],
[                                  0, 0, 0,                                  0],
[                                  0, 0, 0,                                  0],
[4*G*M*(-2*G*M + c**2*r)/(c**4*r**4), 0, 0,                                  0]])

Matrix([
[0,                             0, 0,                              0],
[0,                             0, 0, 4*G*M/(r**2*(-2*G*M + c**2*r))],
[0,                             0, 0,                              0],
[0, 4*G*M/(r**2*(2*G*M - c**2*r)), 0,                              0]])

Matrix([
[0, 0,              0,               0],
[0, 0,              0,               0],
[0, 0,              0, -8*G*M/(c**2*r)],
[0, 0, 8*G*M/(c**2*r),               0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

## Ricci Tensor

Now we can finally calculate the Ricci tensor. Spoiler alert: Since the Schwarzschild is a vacuum solution, all components of the Ricci tensor must vanish. 
$$ R_{ij} = R^k_{ikj}$$

In [179]:
def Ricci(i,j):
    ricc = 0
    n = 0
    for k in range(rank):
        ricc = ricc + Rie[k][i][k,j]
        r = ricc.simplify()
    return r

In [180]:
Ric = [Ricci(i,j) for i in range(rank) for j in range(rank)]

In [181]:
Ric

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Invariante de Kretschmann

In order to verify the nature of the divergence, we need to calculate a metric invariant, in particular, the Kretschmann invariant which is defined as

$$ K = R^{m_0 n_0 p_0 q_0} R_{m_0 n_0 p_0 q_0} = g_{m_0 m_1} g^{n_0 n_1} g^{p_0 p_1} g^{q_0 q_1} R^{m_0}_{n_1 p_1 q_1} R^{m_1}_{n_0 p_0 q_0} $$

In [194]:
indices = list(product([0,1,2,3], repeat=2))
def Kr():
    K = 0
    for m in indices:
        for n in indices:
            for p in indices: 
                for q in indices:
                    K += g[m[0],m[1]] * gI[n[0],n[1]] * gI[p[0],p[1]] * gI[q[0],q[1]] * Rie[m[0]][n[1]][p[1], q[1]] * Rie[m[1]][n[0]][p[0],q[0]]
    K = K.simplify() / 16 # Esse 16 corrige as redundancias que o 'for loop' gera nessas contas
    return K

In [195]:
Kr()

48*G**2*M**2/(c**4*r**6)